In [54]:
import fastf1 as ff1
import pandas as pd

# Load the race session for the 2018 Australian GP
race = ff1.get_session(2019, 'German Grand Prix', 'R')

# Load the session data (this is necessary for laps and telemetry)
race.load()

# Access all laps for the race using the laps attribute
laps = race.laps

# Extract driver, lap number, sector times, and the total lap time for each lap and driver
sector_times_df = laps[['Driver', 'LapNumber', 'Sector1Time', 'Sector2Time', 'Sector3Time', 'LapTime']]

core           INFO 	Loading data for German Grand Prix - Race [v3.4.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['33', '5', '26', '18', '55', '23', '8', '20', '44', '88', '63', '7', '99', '10', '77', '27', '16', '4', '3', '11']


In [55]:
sector_times_df

,Driver,LapNumber,Sector1Time,Sector2Time,Sector3Time,LapTime
0,GAS,1.0,NaT,0 days 00:00:47.155000,0 days 00:00:33.410000,0 days 00:01:51.390000
1,GAS,2.0,0 days 00:00:22.479000,0 days 00:00:44.297000,0 days 00:00:37.417000,0 days 00:01:44.193000
2,GAS,3.0,0 days 00:00:26.604000,0 days 00:00:53.790000,0 days 00:00:41.221000,0 days 00:02:01.615000
3,GAS,4.0,0 days 00:00:48.870000,0 days 00:00:53.513000,0 days 00:00:37.264000,0 days 00:02:19.647000
4,GAS,5.0,0 days 00:00:22.405000,0 days 00:00:44.690000,0 days 00:00:32.721000,0 days 00:01:39.816000
...,...,...,...,...,...,...
1055,GIO,60.0,0 days 00:00:17.835000,0 days 00:00:37.469000,0 days 00:00:25.603000,0 days 00:01:20.907000
1056,GIO,61.0,0 days 00:00:17.325000,0 days 00:00:37.350000,0 days 00:00:24.182000,0 days 00:01:18.857000
1057,GIO,62.0,0 days 00:00:17.204000,0 days 00:00:36.451000,0 days 00:00:23.852000,0 days 00:01:17.507000
1058,GIO,63.0,0 days 00:00:17.224000,0 days 00:00:36.987000,0 days 00:00:23.848000,0 days 00:01:18.059000


In [56]:
# Update Sector1Time for LapNumber == 1 using LapTime - Sector2Time - Sector3Time
# In this dataset for some reason there is no information for the first sectortime.
condition = sector_times_df['LapNumber'] == 1
sector_times_df.loc[condition, 'Sector1Time'] = sector_times_df.loc[condition, 'LapTime'] - \
                                                 sector_times_df.loc[condition, 'Sector2Time'] - \
                                                 sector_times_df.loc[condition, 'Sector3Time']

# For some reason there where no information for this observation, therefore it was calculated.
condition2 = (sector_times_df['LapNumber'] == 27.0) & (sector_times_df['Driver'] == 'GRO')
sector_times_df.loc[condition2, 'LapTime'] = sector_times_df.loc[condition2, 'Sector3Time'] + \
                                                 sector_times_df.loc[condition2, 'Sector1Time'] + \
                                                 sector_times_df.loc[condition2, 'Sector2Time']


condition3 = (sector_times_df['LapNumber'] == 29.0) & (sector_times_df['Driver'] == 'HAM')
sector_times_df.loc[condition3, 'LapTime'] = sector_times_df.loc[condition3, 'Sector3Time'] + \
                                                 sector_times_df.loc[condition3, 'Sector1Time'] + \
                                                 sector_times_df.loc[condition3, 'Sector2Time']

condition4 = (sector_times_df['LapNumber'] == 30.0) & (sector_times_df['Driver'] == 'HAM')
sector_times_df.loc[condition4, 'LapTime'] = sector_times_df.loc[condition4, 'Sector3Time'] + \
                                                 sector_times_df.loc[condition4, 'Sector1Time'] + \
                                                 sector_times_df.loc[condition4, 'Sector2Time']

condition5 = (sector_times_df['LapNumber'] == 26.0) & (sector_times_df['Driver'] == 'STR')
sector_times_df.loc[condition5, 'LapTime'] = sector_times_df.loc[condition5, 'Sector3Time'] + \
                                                 sector_times_df.loc[condition5, 'Sector1Time'] + \
                                                 sector_times_df.loc[condition5, 'Sector2Time']


# Create dummy columns to indicate the fastest times for each sector and lap time
sector_times_df['FastestSector1'] = (sector_times_df['Sector1Time'] == sector_times_df.groupby('Driver')['Sector1Time'].transform('min')).astype(int)
sector_times_df['FastestSector2'] = (sector_times_df['Sector2Time'] == sector_times_df.groupby('Driver')['Sector2Time'].transform('min')).astype(int)
sector_times_df['FastestSector3'] = (sector_times_df['Sector3Time'] == sector_times_df.groupby('Driver')['Sector3Time'].transform('min')).astype(int)
sector_times_df['FastestLap'] = (sector_times_df['LapTime'] == sector_times_df.groupby('Driver')['LapTime'].transform('min')).astype(int)


# Sort the data by Driver and LapNumber for better readability
sector_times_df = sector_times_df.sort_values(by=['Driver', 'LapNumber']).reset_index(drop=True)
sector_times_df = sector_times_df.fillna(0)

C:\Users\Pc\AppData\Local\Temp\ipykernel_7596\1139352549.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sector_times_df['FastestSector1'] = (sector_times_df['Sector1Time'] == sector_times_df.groupby('Driver')['Sector1Time'].transform('min')).astype(int)
C:\Users\Pc\AppData\Local\Temp\ipykernel_7596\1139352549.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sector_times_df['FastestSector2'] = (sector_times_df['Sector2Time'] == sector_times_df.groupby('Driver')['Sector2Time'].transform('min')).asty

In [57]:
# There are still some NA's for people who did not finish a round.
# i dont know if our plots can handle it NA's however in our case these missing observations actually carry some information.

# Get missing values.
nat_rows = sector_times_df[
    sector_times_df[['Sector1Time', 'Sector2Time', 'Sector3Time', 'LapTime']].isna().any(axis=1)
]

# Count the number of times each driver appears (i.e., number of laps per driver)
driver_lap_counts = sector_times_df['Driver'].value_counts()
print(nat_rows[['Driver','LapNumber','Sector1Time', 'Sector2Time', 'Sector3Time', 'LapTime']])
print(driver_lap_counts)

Empty DataFrame
Columns: [Driver, LapNumber, Sector1Time, Sector2Time, Sector3Time, LapTime]
Index: []
Driver
VER    65
ALB    64
GRO    64
GIO    64
STR    64
VET    64
KUB    64
HAM    64
MAG    64
SAI    64
RAI    64
KVY    64
RUS    64
GAS    62
BOT    57
HUL    40
LEC    27
NOR    25
RIC    14
PER     2
Name: count, dtype: int64


In [37]:
# Now get a dataframe which contains data on speed and RPM for each car and driver.
telemetry_data_list = []

# Loop through each lap to extract telemetry data for speed and RPM at each moment
for lap in laps.iterlaps():
    driver_lap = lap[1]  # Get the lap data
    telemetry = driver_lap.get_car_data()  # Get telemetry data for that lap
    
    # Extract speed and RPM at each point in the lap
    for index, row in telemetry.iterrows():
        telemetry_data_list.append({
            'Driver': driver_lap['Driver'],
            'LapNumber': driver_lap['LapNumber'],
            'Time': row['Time'],  # Timestamp of telemetry data
            'Speed': row['Speed'],
            'RPM': row['RPM']
        })

# Convert telemetry data list to a DataFrame
telemetry_df = pd.DataFrame(telemetry_data_list)

In [38]:
telemetry_df = pd.merge(telemetry_df, sector_times_df[['Driver', 'LapNumber', 'Sector1Time', 'Sector2Time', 'Sector3Time']],
                        on=['Driver', 'LapNumber'], 
                        how='left')

telemetry_df

,Driver,LapNumber,Time,Speed,RPM,Sector1Time,Sector2Time,Sector3Time
0,GAS,1.0,0 days 00:00:00.139000,0,10530,0 days 00:00:30.825000,0 days 00:00:47.155000,0 days 00:00:33.410000
1,GAS,1.0,0 days 00:00:00.378000,0,10042,0 days 00:00:30.825000,0 days 00:00:47.155000,0 days 00:00:33.410000
2,GAS,1.0,0 days 00:00:00.658000,4,8082,0 days 00:00:30.825000,0 days 00:00:47.155000,0 days 00:00:33.410000
3,GAS,1.0,0 days 00:00:00.897000,10,6402,0 days 00:00:30.825000,0 days 00:00:47.155000,0 days 00:00:33.410000
4,GAS,1.0,0 days 00:00:01.137000,13,7764,0 days 00:00:30.825000,0 days 00:00:47.155000,0 days 00:00:33.410000
...,...,...,...,...,...,...,...,...
439139,GIO,64.0,0 days 00:01:17.467000,255,11811,0 days 00:00:17.412000,0 days 00:00:37.138000,0 days 00:00:24.074000
439140,GIO,64.0,0 days 00:01:17.707000,258,11957,0 days 00:00:17.412000,0 days 00:00:37.138000,0 days 00:00:24.074000
439141,GIO,64.0,0 days 00:01:17.947000,260,10867,0 days 00:00:17.412000,0 days 00:00:37.138000,0 days 00:00:24.074000
439142,GIO,64.0,0 days 00:01:18.187000,264,10782,0 days 00:00:17.412000,0 days 00:00:37.138000,0 days 00:00:24.074000


In [39]:
import pandas as pd

# Define the function to assign sector dummies
def assign_sector_dummy(row):
    # Convert sector times to total seconds for comparison
    sector1_end = row['Sector1Time']
    sector2_end = sector1_end + row['Sector2Time']
    
    # Convert 'Time' to seconds
    time_in_seconds = row['Time']

    # Initialize dummies
    sector1_dummy = 0
    sector2_dummy = 0
    sector3_dummy = 0

    # Assign sector dummies based on the 'Time'
    if time_in_seconds < sector1_end:
        sector1_dummy = 1
    elif sector1_end <= time_in_seconds < sector2_end:
        sector2_dummy = 1
    else:
        sector3_dummy = 1

    # Return the dummies
    return pd.Series([sector1_dummy, sector2_dummy, sector3_dummy], index=['Sector1', 'Sector2', 'Sector3'])

# Apply the function to the DataFrame to create dummy variables for each observation
telemetry_df[['Sector1', 'Sector2', 'Sector3']] = telemetry_df.apply(assign_sector_dummy, axis=1)



In [40]:
# Function to format the timedelta back to MM:SS.fff
def format_timedelta_back(timedelta_obj):
    if pd.isna(timedelta_obj):
        return None
    total_seconds = timedelta_obj.total_seconds()
    minutes, seconds = divmod(total_seconds, 60)
    return f"{int(minutes):02}:{seconds:06.3f}"

# Apply the formatting function to 'Time', 'Sector1Time', 'Sector2Time', and 'Sector3Time'
telemetry_df['Time'] = telemetry_df['Time'].apply(format_timedelta_back)
telemetry_df['Sector1Time'] = telemetry_df['Sector1Time'].apply(format_timedelta_back)
telemetry_df['Sector2Time'] = telemetry_df['Sector2Time'].apply(format_timedelta_back)
telemetry_df['Sector3Time'] = telemetry_df['Sector3Time'].apply(format_timedelta_back)

In [41]:
# Apply the formatting function to 'Time', 'Sector1Time', 'Sector2Time', and 'Sector3Time'
sector_times_df['LapTime'] = sector_times_df['LapTime'].apply(format_timedelta_back)
sector_times_df['Sector1Time'] = sector_times_df['Sector1Time'].apply(format_timedelta_back)
sector_times_df['Sector2Time'] = sector_times_df['Sector2Time'].apply(format_timedelta_back)
sector_times_df['Sector3Time'] = sector_times_df['Sector3Time'].apply(format_timedelta_back)

In [42]:
telemetry_df

,Driver,LapNumber,Time,Speed,RPM,Sector1Time,Sector2Time,Sector3Time,Sector1,Sector2,Sector3
0,GAS,1.0,00:00.139,0,10530,00:30.825,00:47.155,00:33.410,1,0,0
1,GAS,1.0,00:00.378,0,10042,00:30.825,00:47.155,00:33.410,1,0,0
2,GAS,1.0,00:00.658,4,8082,00:30.825,00:47.155,00:33.410,1,0,0
3,GAS,1.0,00:00.897,10,6402,00:30.825,00:47.155,00:33.410,1,0,0
4,GAS,1.0,00:01.137,13,7764,00:30.825,00:47.155,00:33.410,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
439139,GIO,64.0,01:17.467,255,11811,00:17.412,00:37.138,00:24.074,0,0,1
439140,GIO,64.0,01:17.707,258,11957,00:17.412,00:37.138,00:24.074,0,0,1
439141,GIO,64.0,01:17.947,260,10867,00:17.412,00:37.138,00:24.074,0,0,1
439142,GIO,64.0,01:18.187,264,10782,00:17.412,00:37.138,00:24.074,0,0,1


In [44]:
sector_times_df


,Driver,LapNumber,Sector1Time,Sector2Time,Sector3Time,LapTime,FastestSector1,FastestSector2,FastestSector3,FastestLap
64,BOT,1.0,00:27.609,00:43.559,00:31.026,01:42.194,0,0,0,0
65,BOT,2.0,00:21.789,00:43.216,00:31.270,01:36.275,0,0,0,0
66,BOT,3.0,00:28.260,00:55.551,00:41.371,02:05.182,0,0,0,0
67,BOT,4.0,00:39.478,00:59.898,00:42.668,02:22.044,0,0,0,0
68,BOT,5.0,00:21.833,00:42.015,00:29.228,01:33.076,0,0,0,0
69,BOT,6.0,00:20.743,00:41.400,00:28.858,01:31.001,0,0,0,0
70,BOT,7.0,00:20.527,00:41.619,00:28.579,01:30.725,0,0,0,0
71,BOT,8.0,00:20.228,00:41.509,00:28.247,01:29.984,0,0,0,0
72,BOT,9.0,00:20.258,00:41.204,00:28.223,01:29.685,0,0,0,0
73,BOT,10.0,00:20.007,00:41.215,00:28.260,01:29.482,0,0,0,0
